In [1]:
from cmip6_preprocessing.preprocessing import full_preprocessing

## The oldschool manual way

In [2]:
# Go through the catalogue
import xarray as xr
import pandas as pd
import numpy as np
import warnings
import gcsfs
df = pd.read_csv('https://storage.googleapis.com/pangeo-cmip6/pangeo-cmip6-zarr-consolidated-stores.csv')

pick = dict()
variables = ['thetao', 'uo', 'vo']
for var in variables:
    # pick all available 'piControl' experiements with ocean temperature fields
    pick[var] = df[ (df.variable_id == var) & (df.experiment_id == 'piControl') &
            (df.grid_label == 'gn')
         ]

In [3]:
# all available models
models = np.unique(np.concatenate([p['source_id'].values.flat for p in list(pick.values())]))#

In [4]:
# path_dict = {model:{} for model,uri in zip(pick['source_id'].values, pick['zstore'].values)}
path_dict = {model:{} for model in models}
for var in pick.keys():
    for model, uri in zip(pick[var]['source_id'].values, pick[var]['zstore'].values):
        path_dict[model][var] = uri

        # # this is an unstructured grid...we cant process that yet
del path_dict['AWI-CM-1-1-MR']

# sort out models that have all three variables 
path_dict = {k:v for k,v in path_dict.items() if set(v.keys()) == set(variables)}

path_dict.keys()

dict_keys(['BCC-CSM2-MR', 'BCC-ESM1', 'CAMS-CSM1-0', 'CESM2', 'CESM2-WACCM', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'CanESM5', 'EC-Earth3', 'EC-Earth3-Veg', 'GFDL-CM4', 'GISS-E2-1-G', 'GISS-E2-1-G-CC', 'HadGEM3-GC31-LL', 'MCM-UA-1-0', 'MIROC-ES2L', 'MIROC6', 'NESM3', 'SAM0-UNICON'])

In [ ]:
# now replace the zstore links with xarray datasets
# read the data (lazily) into dictionary
def read_data(uri_dict):
    datasets = {}
    for k,v in uri_dict.items():
#         print("reading: %s" %k)
        gcs = gcsfs.GCSFileSystem(token='anon')
        ds = xr.open_zarr(gcs.get_mapper(v), consolidated=True)
        datasets[k] = ds
    return datasets

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    data_dict = dict()
    for k,v in path_dict.items():
        print(k)
        if k not in ['EC-Earth3-LR']:
            data_dict[k] = read_data(v)

## With intake_esm

In [3]:
# open ESMCol catalog
import intake
col = intake.open_esm_datastore("../../cmip6hack-ocean-bgc/catalogs/pangeo-cmip6.json")

In [4]:
# search and display data
cat = col.search(experiment_id=['piControl'], table_id='Oyr', variable_id=['o2'],
                 grid_label='gn')
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
4283,CMIP,CCCma,CanESM5,piControl,r1i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/piControl/r1i1p1...,NaN
4333,CMIP,CCCma,CanESM5,piControl,r1i1p2f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/piControl/r1i1p2...,NaN
9082,CMIP,IPSL,IPSL-CM6A-LR,piControl,r1i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/IPSL/IPSL-CM6A-LR/piControl/r1...,NaN
9130,CMIP,IPSL,IPSL-CM6A-LR,piControl,r1i2p1f1,Oyr,o2,gn,gs://cmip6/CMIP/IPSL/IPSL-CM6A-LR/piControl/r1...,NaN
9449,CMIP,MIROC,MIROC-ES2L,piControl,r1i1p1f2,Oyr,o2,gn,gs://cmip6/CMIP/MIROC/MIROC-ES2L/piControl/r1i...,NaN


In [5]:
# load to a dictionary of xarray Datasets - may take a minute
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True},
                                cdf_kwargs={'chunks': {'time':20}})
list(dset_dict.keys())

# # look at a particular dataset
# ds = dset_dict['CMIP.CCCma.CanESM5.historical.Oyr.gn']
# ds

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 3 group(s)


/Users/juliusbusecke/miniconda/envs/cmip6-hackathon-demos/lib/python3.7/site-packages/xarray/coding/times.py:459: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/Users/juliusbusecke/miniconda/envs/cmip6-hackathon-demos/lib/python3.7/site-packages/xarray/coding/times.py:459: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/Users/juliusbusecke/miniconda/envs/cmip6-hackathon-demos/lib/python3.7/site-packages/xarray/coding/times.py:459: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf

RecursionError: maximum recursion depth exceeded while calling a Python object